In [64]:
from phasic_tonic.detect_phasic import detect_phasic
from phasic_tonic.DatasetLoader import DatasetLoader
from phasic_tonic.helper import get_metadata
from phasic_tonic.runtime_logger import logger_setup
from phasic_tonic.utils import get_sequences, get_segments

import math
import numpy as np
import pandas as pd
import pynapple as nap

from tqdm.auto import tqdm
from scipy.io import loadmat
from mne.filter import resample

targetFs = 500

logger = logger_setup()

CONFIG_DIR = "/home/nero/phasic_tonic/data/dataset_loading.yaml"
DATASET_DIR = "/home/nero/datasets/preprocessed"

def get_start_end(hypno: np.ndarray, sleep_state_id: int):
    """Convert sleep states into lists of start and end time indices."""
    seq = get_sequences(np.where(hypno == sleep_state_id)[0])
    start = []
    end = []
    for s, e in seq:
        start.append(s)
        end.append(e)
    return (start, end)

def str_to_tuple(string):
    string = string.strip("()")
    parts = string.split(",")
    return tuple(map(int, parts))

def load_data(fname):
    loaded_data = np.load(fname)
    loaded_dict = {str_to_tuple(key): loaded_data[key] for key in loaded_data.files}
    return loaded_dict

def half_round_up(n):
    if n - math.floor(n) < 0.5:
        return math.floor(n)
    return math.ceil(n)

In [90]:
from pathlib import Path
from phasic_tonic.detect_phasic_v2 import detect_phasic_v2

compressed_datasets = list(Path(DATASET_DIR).glob('*.npz'))
container = []

with tqdm(compressed_datasets) as datasets:
    for fname in datasets:
        metaname = str(fname.stem)

        datasets.set_postfix_str(metaname)
        metadata = get_metadata(metaname)

        rem_epochs = load_data(fname)
        print(f"Loaded:{metaname}")

        if not rem_epochs:
            continue

        phREM = detect_phasic_v2(rem_epochs, fs=500)
        
        # Classify each time window as tonic or phasic event
        for rem_idx in phREM:
            # Container for storing results and metadata
            data = metadata.copy()
            
            rem_start, rem_end = rem_idx[0], rem_idx[1]
            data["rem_start"] = rem_start
            data["rem_end"] = rem_end

            phasic_idx = phREM[rem_idx]

            # Fill the whole REM epoch as Tonic state
            result = ["T" for _ in range(rem_end-rem_start)] 

            # print(f"REM epoch {rem_idx}")
            if phasic_idx:
                for ph in phasic_idx:
                    # Round up the phasic timestamp if its fractional part is greater than 0.5
                    ph_start, ph_end = half_round_up(ph[0]/500), half_round_up(ph[1]/500)

                    # print(f"({ph[0]/500}, {ph[1]/500}) -> ({ph_start}, {ph_end})")
                    ph_start -= rem_start
                    ph_end -= rem_start
                    
                    # print(ph_start, ph_end)
                    result[ph_start:ph_end] = "P"
            
            # print("".join(result))
            data["event_strings"] = "".join(result)
            container.append(data)


0it [00:00, ?it/s]

Loaded:Rat2_SD2_OR_2_posttrial5-3
Loaded:Rat4_SD4_HC_3_posttrial4
Loaded:Rat5_SD3_OD_0_posttrial5-1
Loaded:Rat3_SD4_OD_4_posttrial2
Loaded:Rat9_SD16_OR-N_4_posttrial4
Loaded:Rat4_SD2_HC_0_posttrial2
Loaded:Rat4_SD4_OR-N_4_posttrial5-2
Loaded:Rat6_SD3_HC_4_posttrial1
Loaded:Rat4_SD4_OR-N_4_posttrial4
Loaded:Rat4_SD1_OR_0_posttrial5-3
Loaded:Rat3_SD3_OR_3_posttrial5-2
Loaded:Rat6_SD3_CON_2_posttrial5-2
Loaded:Rat9_SD2_CON_4_posttrial5-0
Loaded:Rat5_SD6_HC_1_posttrial2
Loaded:Rat9_SD2_CON_4_posttrial3
Loaded:Rat9_SD4_CON_2_posttrial3
Loaded:Rat3_SD3_OR_3_posttrial4
Loaded:Rat2_SD3_CON_2_posttrial5-2
Loaded:Rat4_SD5_OD_3_posttrial4
Loaded:Rat8_SD1_HC_3_posttrial4
Loaded:Rat1_SD1_OD_4_posttrial2
Loaded:Rat9_SD4_CON_2_posttrial5-1
Loaded:Rat13_SD3_OR_4_posttrial2
Loaded:Rat1_SD2_OD_2_posttrial4
Loaded:Rat9_SD10_OR_4_posttrial4
Loaded:Rat2_SD2_OR_2_posttrial5-1
Loaded:Rat4_SD16_OR_0_posttrial5-3
Loaded:Rat3_SD3_OR_0_posttrial3
Loaded:Rat11_SD2_OD_4_posttrial1
Loaded:Rat9_SD3_OR_2_posttrial5-3

In [87]:
container = pd.DataFrame(container)

[{'rat_id': 2,
  'study_day': 2,
  'condition': 'OR',
  'treatment': 2,
  'trial_num': '5-3',
  'rem_start': 9121,
  'rem_end': 9168,
  'event_strings': 'TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTPTTTTTTTTT'},
 {'rat_id': 2,
  'study_day': 2,
  'condition': 'OR',
  'treatment': 2,
  'trial_num': '5-3',
  'rem_start': 9430,
  'rem_end': 9468,
  'event_strings': 'TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT'},
 {'rat_id': 2,
  'study_day': 2,
  'condition': 'OR',
  'treatment': 2,
  'trial_num': '5-3',
  'rem_start': 9738,
  'rem_end': 9820,
  'event_strings': 'TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTPTTTTTTTTTTTTTTTTTTTPTPTTTTT'}]

In [117]:
container.to_csv("phasic_tonic_strings.csv", index=False)

In [118]:
container

,rat_id,study_day,condition,treatment,trial_num,rem_start,rem_end,event_strings
0,2,2,OR,2,5-3,9121,9168,TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTPTTTTTTTTT
1,2,2,OR,2,5-3,9430,9468,TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT
2,2,2,OR,2,5-3,9738,9820,TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT...
3,4,4,HC,3,4,2396,2400,TTTT
4,4,4,HC,3,4,2411,2419,TTTTTTTT
...,...,...,...,...,...,...,...,...
2122,1,1,OD,4,5-2,7379,7428,TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT
2123,1,1,OD,4,5-2,7850,7866,TTTTTTTTTTTTTTTT
2124,1,1,OD,4,5-2,8002,8034,TTTTTTTTTPTTTTTTTTTTTTTTTTTTTTTT
2125,5,3,OD,0,5-0,1866,1943,TTTTTTTTTTTTTTPTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT...


In [120]:
for event_string in container['event_strings']:
    if 'PP' in event_string:
        print(event_string)

TTTTTTTTTTTTTTTTTPTTTTTTTTTTTTTTTTTTTTTTTPTTTTTPTTTTTTTPPTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT
TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTPPTTTTTTTTTTTTTTTPTTTTTTTTPTTTTTTTTTTTPPTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTPTTTTTTTTTTTTTTTTTTT
TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTPPTT
TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTPTTTTTTTTTTTTTTTTTTTPTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTPPTTTTTTTPTTTTTTTTTTTTTTTTTTTTTTTT
TTTTTTTTTTTTTTTTTTTTTTTTTTTTTPPTTTTTTTTTTTTTTTTTTTPTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTPTTTTTTTTTTTTTTTTTTTTPTTTTTTTTTTTTTTPTTTTTPTT
TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTPTTTTTTTTTTTTTTPPTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTPTTPTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT
TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTPTTTTTTTTTTTTTTTPPTTTT
TTTTTTTTTTTTTTPPTTTTTTTTTTTTTTTTTTTTTTT

In [121]:
# Classify each time window as tonic or phasic event
for rem_idx in phREM:
    rem_start, rem_end = rem_idx[0], rem_idx[1]
    phasic_idx = phREM[rem_idx]

    # Fill the whole REM epoch as Tonic state
    result = ["T" for _ in range(rem_end-rem_start)]

    print(f"REM epoch {rem_idx}")
    if phasic_idx:
        for ph in phasic_idx:
            # Round up the phasic timestamp if its fractional part is greater than 0.5
            ph_start, ph_end = half_round_up(ph[0]/500), half_round_up(ph[1]/500)
            print(f"({ph[0]/500}, {ph[1]/500}) -> ({ph_start}, {ph_end})")

            ph_start -= rem_start
            ph_end -= rem_start
            print(ph_start, ph_end)
            
            result[ph_start:ph_end] = "P"
    
    print("".join(result))

REM epoch (2305, 2443)
(2310.458, 2313.976) -> (2310, 2314)
5 9
(2337.328, 2340.446) -> (2337, 2340)
32 35
TTTTTPTTTTTTTTTTTTTTTTTTTTTTTTTTPTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT


In [ ]:
2310,2311

Check the number of loaded recordings

In [5]:
cbd_cnt = 0
rgs_cnt = 0
os_cnt = 0

# Count recordings belonging to CBD dataset
for fname in datasets:
    metadata = get_metadata(fname.stem)
    if metadata['treatment'] == 0 or metadata['treatment'] == 1:
        cbd_cnt += 1
    elif metadata['treatment'] == 2 or metadata['treatment'] == 3:
        rgs_cnt += 1
    elif metadata['treatment'] == 4:
        os_cnt += 1

assert cbd_cnt == 170
assert rgs_cnt == 159
assert os_cnt == 210

AssertionError: 